• Prepare a structured dataset that includes the following fields, where available:
o Company name: Name of the company
o Country: country in which the company has its main headquarters
o Industry: Industry classification based on your source
o Year(s): Year(s) associated with the financial value; please extract the most recent 3
years of the companies’ financials, if available
o Revenue: Revenue figure
o Revenue unit: Unit or currency of the revenue
o (Optional: Add 3–5 additional KPIs of your choice in the same manner as for Revenue.)
• Please ensure that the dataset contains at least 100 companies and no more than 500
companies.

In [109]:
#!pip install kaggle

In [110]:
#!pip install kagglehub[pandas-datasets]


Before the next step I set up the kaggle API with my token:

export KAGGLE_API_TOKEN=<--Token-->

kaggle competitions list  // this helped me check if the api was working

In [111]:
#!kaggle datasets download -d rm1000/fortune-500-companies

In [112]:
#!unzip fortune-500-companies.zip

Firsy Look at the database

In [113]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

file_path = "/home/nathalia-uribe/Documentos/NATHY 2.0/LEARNING/Networks/Ejercicios Libro/Fortune 500 Companies.csv"

fortune500df = pd.read_csv(file_path)

print("First 5 records:", fortune500df.head())

First 5 records:                          name  rank  year                industry sector  \
0  General Motors Corporation     1  1996  Motor Vehicles & Parts    NaN   
1          Ford Motor Company     2  1996  Motor Vehicles & Parts    NaN   
2           Exxon Corporation     3  1996      Petroleum Refining    NaN   
3       Wal-Mart Stores, Inc.     4  1996   General Merchandisers    NaN   
4                  AT&T Corp.     5  1996      Telecommunications    NaN   

  headquarters_state headquarters_city  market_value_mil  revenue_mil  \
0                 MI               NaN               NaN     168828.6   
1                 MI               NaN               NaN     137137.0   
2                 TX               NaN               NaN     110009.0   
3                 AR               NaN               NaN      93627.0   
4                 NY               NaN               NaN      79609.0   

   profit_mil  asset_mil  employees founder_is_ceo female_ceo  \
0         NaN        N

Checking what metrics are easier to obtain

In [114]:
fortune500df.isnull().mean()


name                       0.000000
rank                       0.000000
year                       0.000000
industry                   0.000000
sector                     0.677188
headquarters_state         0.000000
headquarters_city          0.462339
market_value_mil           0.628049
revenue_mil                0.000000
profit_mil                 0.605667
asset_mil                  0.605452
employees                  0.677188
founder_is_ceo             0.677188
female_ceo                 0.677188
newcomer_to_fortune_500    0.677188
global_500                 0.713056
dtype: float64

Dropping every column that is not needed for the assignment

In [115]:
f500_clean = fortune500df.drop(columns=['founder_is_ceo', 'female_ceo', 'newcomer_to_fortune_500', 'global_500','rank'])

In [116]:
f500_clean.head()

,name,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,asset_mil,employees
0,General Motors Corporation,1996,Motor Vehicles & Parts,NaN,MI,NaN,NaN,168828.6,NaN,NaN,NaN
1,Ford Motor Company,1996,Motor Vehicles & Parts,NaN,MI,NaN,NaN,137137.0,NaN,NaN,NaN
2,Exxon Corporation,1996,Petroleum Refining,NaN,TX,NaN,NaN,110009.0,NaN,NaN,NaN
3,"Wal-Mart Stores, Inc.",1996,General Merchandisers,NaN,AR,NaN,NaN,93627.0,NaN,NaN,NaN
4,AT&T Corp.,1996,Telecommunications,NaN,NY,NaN,NaN,79609.0,NaN,NaN,NaN


Checking how many unique companies I have

In [117]:
f500_clean['name'].nunique()

2255

Dropping more companies by suitability of the name, i deleted every name that is numeric only, is punctuation only, is too short (like single letters).

In [118]:
f500_clean = f500_clean.dropna(subset=['name'])

valid_names= r'^[A-Za-z]{2,}.*$'
f500_clean = f500_clean[f500_clean['name'].str.match(valid_names, na=False)]


Checking how many companies I still have...

In [119]:
f500_clean['name'].nunique()

2187

In [120]:


f500_clean.head()

,name,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,asset_mil,employees
0,General Motors Corporation,1996,Motor Vehicles & Parts,NaN,MI,NaN,NaN,168828.6,NaN,NaN,NaN
1,Ford Motor Company,1996,Motor Vehicles & Parts,NaN,MI,NaN,NaN,137137.0,NaN,NaN,NaN
2,Exxon Corporation,1996,Petroleum Refining,NaN,TX,NaN,NaN,110009.0,NaN,NaN,NaN
3,"Wal-Mart Stores, Inc.",1996,General Merchandisers,NaN,AR,NaN,NaN,93627.0,NaN,NaN,NaN
4,AT&T Corp.,1996,Telecommunications,NaN,NY,NaN,NaN,79609.0,NaN,NaN,NaN


Adding the count of years with market value ...

In [121]:
f500_clean['num_years'] = f500_clean.groupby('name')['year'].transform('nunique')

Ordering by name and years , so i see them in order

In [122]:
f500_clean.sort_values(["name", "year"], ascending=[True, False])


,name,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,asset_mil,employees,num_years
13914,ABM Industries,2023,Diversified Outsourcing Services,Business Services,NY,New York,2971.0,7807.0,230.0,4869.0,127000.0,7
12901,ABM Industries,2021,Diversified Outsourcing Services,Business Services,NY,New York,3422.0,5988.0,0.0,3777.0,114000.0,7
12401,ABM Industries,2020,Diversified Outsourcing Services,Business Services,NY,New York,1623.0,6499.0,127.0,3693.0,140000.0,7
11902,ABM Industries,2019,Diversified Outsourcing Services,Business Services,NY,New York,2408.0,6442.0,98.0,3628.0,140000.0,7
11437,ABM Industries,2018,Diversified Outsourcing Services,Business Services,NY,New York,2200.0,5454.0,4.0,3813.0,140000.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
11224,salesforce.com,2018,Computer Software,Technology,CA,San Francisco,85074.0,10480.0,128.0,21010.0,29000.0,5
10765,salesforce.com,2017,Computer Software,Technology,CA,San Francisco,58362.0,8392.0,180.0,17585.0,25000.0,5
10325,salesforce.com,2016,Computer Software,Technology,CA,San Francisco,49533.0,6667.0,47.0,12771.0,19742.0,5
9922,salesforce.com,2015,Computer Software,Technology,CA,San Francisco,43467.0,5374.0,-263.0,10693.0,16200.0,5


Creating a copy of the table without companies that have less than 2 years.....

In [123]:
f500_clean= f500_clean[f500_clean['num_years'] >= 3].copy()

Checking how many companies I still have...

In [124]:
f500_clean['name'].nunique()

1525

In [125]:
f500_clean.sort_values(["name", "year"], ascending=[True, False])

,name,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,asset_mil,employees,num_years
13914,ABM Industries,2023,Diversified Outsourcing Services,Business Services,NY,New York,2971.0,7807.0,230.0,4869.0,127000.0,7
12901,ABM Industries,2021,Diversified Outsourcing Services,Business Services,NY,New York,3422.0,5988.0,0.0,3777.0,114000.0,7
12401,ABM Industries,2020,Diversified Outsourcing Services,Business Services,NY,New York,1623.0,6499.0,127.0,3693.0,140000.0,7
11902,ABM Industries,2019,Diversified Outsourcing Services,Business Services,NY,New York,2408.0,6442.0,98.0,3628.0,140000.0,7
11437,ABM Industries,2018,Diversified Outsourcing Services,Business Services,NY,New York,2200.0,5454.0,4.0,3813.0,140000.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...
11679,salesforce.com,2019,Computer Software,Technology,CA,San Francisco,122103.0,13282.0,1110.0,30737.0,35000.0,5
11224,salesforce.com,2018,Computer Software,Technology,CA,San Francisco,85074.0,10480.0,128.0,21010.0,29000.0,5
10765,salesforce.com,2017,Computer Software,Technology,CA,San Francisco,58362.0,8392.0,180.0,17585.0,25000.0,5
10325,salesforce.com,2016,Computer Software,Technology,CA,San Francisco,49533.0,6667.0,47.0,12771.0,19742.0,5


Ordering my table and dropping the bottom 88 companies with less years .... Since I have 588 companies with all the information I need , then I'll drop the ones that have been on the list for the least years 

In [126]:
f500_clean = f500_clean.sort_values(by='num_years', ascending=False)

company_years = f500_clean[['name', 'num_years']].drop_duplicates()
companies_to_drop = company_years.sort_values(by='num_years').head(1025)['name']
f500_clean = f500_clean[~f500_clean['name'].isin(companies_to_drop)].copy()
f500_clean.head()

,name,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,asset_mil,employees,num_years
13479,Johnson & Johnson,2023,Pharmaceuticals,Health Care,NJ,New Brunswick,483576.00,94943.00,17941.0,187378.00,152700.0,28
2527,Johnson & Johnson,2001,Pharmaceuticals,NaN,NJ,NaN,NaN,29139.00,NaN,NaN,NaN,28
8651,Edison International,2013,Utilities: Gas and Electric,NaN,CA,NaN,1639.48,131.04,-183.0,443.94,NaN,28
3009,Johnson & Johnson,2002,Pharmaceuticals,NaN,NJ,NaN,NaN,33004.00,NaN,NaN,NaN,28
612,Abbott Laboratories,1997,Pharmaceuticals,NaN,IL,NaN,NaN,11013.50,NaN,NaN,NaN,28


In [127]:
f500_clean['name'].nunique()

500

Adding extra columns , as requested with the currency/ metrics  AND also country since the data set I found had the city of headquarters but not the country. The state and city were left intentionally... for clarity, that ineed all headquarters are in the US.

In [128]:
f500_clean['country'] = 'USA'
f500_clean['market_value_currency'] = 'USD'
f500_clean['revenue_currency'] = 'USD'
f500_clean['profit_currency'] = 'USD'
f500_clean['asset_currency'] = 'USD'
f500_clean['employees_metric'] = 'employees'
f500_clean.sort_values(["name", "year"], ascending=[True, False])

,name,year,industry,sector,headquarters_state,headquarters_city,market_value_mil,revenue_mil,profit_mil,asset_mil,employees,num_years,country,market_value_currency,revenue_currency,profit_currency,asset_currency,employees_metric
13766,AES,2023,Utilities: Gas and Electric,Energy,VA,Arlington,16110.0,12617.0,546.0,38363.0,9100.0,9,USA,USD,USD,USD,USD,employees
13272,AES,2022,Utilities: Gas and Electric,Energy,VA,Arlington,17172.0,11141.0,409.0,32963.0,8450.0,9,USA,USD,USD,USD,USD,employees
12752,AES,2021,Utilities: Gas and Electric,Energy,VA,Arlington,18180.0,9660.0,46.0,34603.0,8162.0,9,USA,USD,USD,USD,USD,employees
12249,AES,2020,Utilities: Gas and Electric,Energy,VA,Arlington,9041.0,10189.0,303.0,33648.0,8000.0,9,USA,USD,USD,USD,USD,employees
11735,AES,2019,Utilities: Gas and Electric,Energy,VA,Arlington,11975.0,10736.0,1203.0,32521.0,9000.0,9,USA,USD,USD,USD,USD,employees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5706,"Yum! Brands, Inc.",2007,Food Services,NaN,KY,Louisville,NaN,9561.0,NaN,NaN,NaN,12,USA,USD,USD,USD,USD,employees
5202,"Yum! Brands, Inc.",2006,Food Services,NaN,KY,NaN,NaN,9349.0,NaN,NaN,NaN,12,USA,USD,USD,USD,USD,employees
4696,"Yum! Brands, Inc.",2005,Food Services,NaN,KY,NaN,NaN,9011.0,NaN,NaN,NaN,12,USA,USD,USD,USD,USD,employees
4196,"Yum! Brands, Inc.",2004,Food Services,NaN,KY,NaN,NaN,8380.0,NaN,NaN,NaN,12,USA,USD,USD,USD,USD,employees


In [129]:
f500_clean['name'].nunique()

500

Ordering the columns

In [130]:
f500_clean = f500_clean[
    ['name','num_years','year','industry','sector','country','headquarters_state','headquarters_city',
     'market_value_mil','market_value_currency','revenue_mil','revenue_currency',
     'profit_mil','profit_currency','asset_mil','asset_currency','employees',
     'employees_metric']
]

f500_clean.sort_values(["name", "year"], ascending=[True, False])


,name,num_years,year,industry,sector,country,headquarters_state,headquarters_city,market_value_mil,market_value_currency,revenue_mil,revenue_currency,profit_mil,profit_currency,asset_mil,asset_currency,employees,employees_metric
13766,AES,9,2023,Utilities: Gas and Electric,Energy,USA,VA,Arlington,16110.0,USD,12617.0,USD,546.0,USD,38363.0,USD,9100.0,employees
13272,AES,9,2022,Utilities: Gas and Electric,Energy,USA,VA,Arlington,17172.0,USD,11141.0,USD,409.0,USD,32963.0,USD,8450.0,employees
12752,AES,9,2021,Utilities: Gas and Electric,Energy,USA,VA,Arlington,18180.0,USD,9660.0,USD,46.0,USD,34603.0,USD,8162.0,employees
12249,AES,9,2020,Utilities: Gas and Electric,Energy,USA,VA,Arlington,9041.0,USD,10189.0,USD,303.0,USD,33648.0,USD,8000.0,employees
11735,AES,9,2019,Utilities: Gas and Electric,Energy,USA,VA,Arlington,11975.0,USD,10736.0,USD,1203.0,USD,32521.0,USD,9000.0,employees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5706,"Yum! Brands, Inc.",12,2007,Food Services,NaN,USA,KY,Louisville,NaN,USD,9561.0,USD,NaN,USD,NaN,USD,NaN,employees
5202,"Yum! Brands, Inc.",12,2006,Food Services,NaN,USA,KY,NaN,NaN,USD,9349.0,USD,NaN,USD,NaN,USD,NaN,employees
4696,"Yum! Brands, Inc.",12,2005,Food Services,NaN,USA,KY,NaN,NaN,USD,9011.0,USD,NaN,USD,NaN,USD,NaN,employees
4196,"Yum! Brands, Inc.",12,2004,Food Services,NaN,USA,KY,NaN,NaN,USD,8380.0,USD,NaN,USD,NaN,USD,NaN,employees


In [137]:
f500_clean.to_csv('f500clean.csv', index=False)

Filtering the years to keep the 3 most recent only

In [131]:
Final_Fortune_500 = f500_clean.groupby('name', group_keys=False).apply(lambda x: x.nlargest(3, 'year'))
Final_Fortune_500.sort_values(["name", "year"], ascending=[True, False])

/tmp/ipykernel_41264/3197569287.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Final_Fortune_500 = f500_clean.groupby('name', group_keys=False).apply(lambda x: x.nlargest(3, 'year'))


,name,num_years,year,industry,sector,country,headquarters_state,headquarters_city,market_value_mil,market_value_currency,revenue_mil,revenue_currency,profit_mil,profit_currency,asset_mil,asset_currency,employees,employees_metric
13766,AES,9,2023,Utilities: Gas and Electric,Energy,USA,VA,Arlington,16110.00,USD,12617.00,USD,546.0,USD,38363.00,USD,9100.0,employees
13272,AES,9,2022,Utilities: Gas and Electric,Energy,USA,VA,Arlington,17172.00,USD,11141.00,USD,409.0,USD,32963.00,USD,8450.0,employees
12752,AES,9,2021,Utilities: Gas and Electric,Energy,USA,VA,Arlington,18180.00,USD,9660.00,USD,46.0,USD,34603.00,USD,8162.0,employees
9201,AGCO Corporation,13,2014,Construction and Farm Machinery,NaN,USA,GA,NaN,5167.00,USD,10787.00,USD,597.0,USD,8439.00,USD,NaN,employees
8711,AGCO Corporation,13,2013,Construction and Farm Machinery,NaN,USA,GA,NaN,505.75,USD,996.22,USD,522.1,USD,772.18,USD,NaN,employees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,York International Corporation,10,2004,Industrial and Farm Equipment,NaN,USA,PA,NaN,NaN,USD,4076.10,USD,NaN,USD,NaN,USD,NaN,employees
3871,York International Corporation,10,2003,Industrial and Farm Equipment,NaN,USA,PA,NaN,NaN,USD,3843.40,USD,NaN,USD,NaN,USD,NaN,employees
9155,"Yum! Brands, Inc.",12,2014,Food Services,NaN,USA,KY,NaN,33392.00,USD,13084.00,USD,1091.0,USD,8695.00,USD,NaN,employees
8640,"Yum! Brands, Inc.",12,2013,Food Services,NaN,USA,KY,NaN,3242.55,USD,136.33,USD,1597.0,USD,90.11,USD,NaN,employees


In [132]:
Final_Fortune_500['name'].nunique()

500

Grouping the rows by 'name', 'num_years', 'industry', 'sector', 'country', 'headquarters_state', 'headquarters_city'.

And here is the result:

In [133]:
Final_Fortune_500.isnull().mean()

name                     0.000000
num_years                0.000000
year                     0.000000
industry                 0.000000
sector                   0.708000
country                  0.000000
headquarters_state       0.000000
headquarters_city        0.379333
market_value_mil         0.432000
market_value_currency    0.000000
revenue_mil              0.000000
revenue_currency         0.000000
profit_mil               0.390667
profit_currency          0.000000
asset_mil                0.390667
asset_currency           0.000000
employees                0.708000
employees_metric         0.000000
dtype: float64

In [134]:
Final_Fortune_500['name'].nunique()

500

In [135]:
Final_Fortune_500.to_csv('final_f500.csv', index=False)